# [LEGALST-123] Lab 05: Central Limit Theorem, Confidence Intervals, Hypothesis Testing

In [7]:
from datascience import *
from collections import Counter
import numpy as np
import pandas as pd
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Introduction
In this lab, we aim to prepare students for prediction exercises in PSET 1 and PSET 2 by allowing students to contextualize the statistical ideas of the Central Limit Theorem and hypothesis testing by using a dataset containing continuous variables. 

**Data & EDA**: 
... explanation needed ... 

**Bootsrapping**:
... explanation needed ... 

**Hypothesis Testing**
... explanation needed ... 

**Confidence Interval**: 
... explanation needed ... 

**Central Limit Theorem**: 
... explanation needed ... 

<br/>

<hr style="border: 1px solid #fdb515;" />

## Data

For this lab, we'll be using the same datasets used in our previous labs, the Nashville's police stops dataset. Run the following cell below to read the `DataFrame`.

In [8]:
# Issue loading; Will work in Datahub.
# stops = pd.read_csv("https://github.com/ds-modules/data/raw/main/nashville_sample.csv")

user = "suminpark" #insert your user name
path = "/Users/" + user +  "/Documents/GitHub/Modules/Legalst-123/labs/data" #edit this later for the actual lab. 
# user = "fionaaaaa_mei" #insert your user name
# path = "/Users/" + user +  "/Documents/GitHub/Legalst-123/labs/data" #edit this later for the actual lab. 
stops = pd.read_csv(path + "/stops_sample.csv", index_col = 0)
stops.head()

,index,raw_row_number,date,time,location,lat,lng,precinct,reporting_area,zone,...,raw_traffic_citation_issued,raw_misd_state_citation_issued,raw_suspect_ethnicity,raw_driver_searched,raw_passenger_searched,raw_search_consent,raw_search_arrest,raw_search_warrant,raw_search_inventory,raw_search_plain_view
0,1840907,93347,2010-04-18,13140.0,"BURGESS AVE & WHITE BRIDGE PIKE, NASHVILLE, TN...",36.145004,-86.857970,1.0,5103.0,113.0,...,False,NaN,N,False,False,False,False,False,False,False
1,492044,2001428,2015-01-19,19920.0,"DUE WEST AVE W & S GRAYCROFT AVE, MADISON, TN,...",36.249187,-86.734459,7.0,1797.0,723.0,...,False,False,N,False,False,False,False,False,False,False
2,431170,1996331,2015-01-15,1020.0,"S GALLATIN PIKE & MADISON BLVD, MADISON, TN, 3...",36.254979,-86.715246,7.0,1623.0,711.0,...,False,False,N,False,False,False,False,False,False,False
3,2066423,1319451,2013-05-17,62760.0,"CHARLOTTE PIKE & W HILLWOOD DR, NASHVILLE, TN,...",36.139093,-86.880533,1.0,5009.0,123.0,...,False,False,N,False,False,False,False,False,False,False
4,2899480,201349,2010-09-01,28140.0,"BELL RD & DODSON CHAPEL RD, HERMITAGE, TN, 37076",36.163310,-86.613147,5.0,9501.0,521.0,...,False,NaN,N,False,False,False,False,False,False,False


## 1. Exploratory Data Analysis

Let's refer back to our last lab, when we explored different distributions using histogram. In particular, we looked at distribution of stops counts for `"subject_sex"` and `"subject_age"`. For this notebook, let's look at the distribution of **FIND WHAT SUBJECT WE WANT TO FIND**!!!

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.1**:
Before explore these variables, let's clean the dataset. In the code cell below, drop any columns that has "raw" in their column names. Then, drop the rows with *any* null values EXCEPT for the columns` "contraband_found"`, `"contraband_drugs"`, `"contraband_weapons"`, `"search_basis"`, and `"notes"`.
</div>

Hint 1: Look at lab 04 question 1.1 and question 1.2! It should be very similar. 

In [9]:
# YOUR ANSWER HERE. You can use less or more lines than provided below. 
# ... 
# ... 
# ...
# stops = ...

# stops.head(5)

In [23]:
#SOLUTION 
raw_columns = [col for col in stops.columns if 'raw_' in col]
necessary_cols = stops.columns.drop(["contraband_found", "contraband_drugs", "contraband_weapons", "search_basis", "notes"]) #SOLUTION 
stops = stops.dropna(subset=necessary_cols,axis=0)
stops = stops.drop(raw_columns, axis=1)

stops.head(5)

,index,date,time,location,lat,lng,precinct,reporting_area,zone,subject_age,...,contraband_drugs,contraband_weapons,frisk_performed,search_conducted,search_person,search_vehicle,search_basis,reason_for_stop,vehicle_registration_state,notes
1,492044,2015-01-19,19920.0,"DUE WEST AVE W & S GRAYCROFT AVE, MADISON, TN,...",36.249187,-86.734459,7.0,1797.0,723.0,45.0,...,NaN,NaN,False,False,False,False,NaN,vehicle equipment violation,TN,tail light out
2,431170,2015-01-15,1020.0,"S GALLATIN PIKE & MADISON BLVD, MADISON, TN, 3...",36.254979,-86.715246,7.0,1623.0,711.0,21.0,...,NaN,NaN,False,False,False,False,NaN,moving traffic violation,TN,NaN
3,2066423,2013-05-17,62760.0,"CHARLOTTE PIKE & W HILLWOOD DR, NASHVILLE, TN,...",36.139093,-86.880533,1.0,5009.0,123.0,35.0,...,NaN,NaN,False,False,False,False,NaN,vehicle equipment violation,TN,NaN
5,543402,2012-12-13,67560.0,"3RD AVE S & ASH ST, NASHVILLE, TN, 37210",36.152786,-86.770562,4.0,4005.0,425.0,26.0,...,NaN,NaN,False,False,False,False,NaN,moving traffic violation,TN,RAN STOP SIGN
6,1446083,2012-02-26,55380.0,"BELL RD & DODSON CHAPEL RD, HERMITAGE, TN, 37076",36.163310,-86.613147,5.0,9501.0,521.0,18.0,...,NaN,NaN,False,False,False,False,NaN,moving traffic violation,TN,NaN


Let's explore the column `"subject_race"` and `"subject_age"`. For convenience, we have provided a code below showing what race/ethincity categories exists within the column  `"subject_race"`. For this particular question, let's look at the distribution of people who are categorized as `'hispanic'` or `'white'` for the `"subject_race"`. 

In [22]:
stops["subject_race"].unique()

array(['black', 'white', 'hispanic', 'unknown', 'asian/pacific islander',
       'other'], dtype=object)

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.2**:

Write code that plots a histogram of age distribution specifically for people who are categorized as Hispanic. Assign `subject_hispanic` to the table manipulation you do before plotting histogram. 

</div>

In [24]:
# YOUR CODE HERE

# subject_hispanic = ... 

# Histogram

**What is the shape of the plot?**

_YOUR ANSWER HERE_

**Find the 25th, 50th, 75th percentile below.**

In [27]:
# YOUR CODE HERE
# Find the 25th percentile

In [ ]:
# Find the 50th percentile

In [30]:
# Find the 75th percentile

**From our histogram and different percentiles, what does this distribution tell you about age distribution of Police stops among Hispanic group?** 

_YOUR ANSWER HERE_

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.3**:

Now, do the same process for people who are categorized as White. Assign `subject_white` to the table manipulation you do before plotting histogram. 

</div>

In [26]:
# YOUR CODE HERE

# subject_white = ... 

# Histogram

**What is the shape of the plot?**

_YOUR ANSWER HERE_

**Find the 25th, 50th, 75th percentile below.**

In [28]:
# YOUR CODE HERE
# Find the 25th percentile

In [29]:
# Find the 50th percentile

**How do the two distributions compare?**

_YOUR ANSWER HERE_

<!-- BEGIN QUESTION -->
<div class=“alert alert-warning”>

#### **Question 1.4**:

Create an overlaid historgram comparing the two distributions.

</div>

In [33]:
# YOUR ANSWER HERE

ONE MORE CONTEXTUAL QUESTION within Q1.4 HERE REGARDING TWO DISTRIBUTION.

## 2. Forming a Hypothesis


Hypothesis testing is a fundamental statistical method used to make inferences about a population based on a sample of data. 

To formulate a hypothesis, researchers typically start with a null hypothesis (H0) that posits no effect or no difference, and an alternative hypothesis (H1) suggesting a specific effect or difference. 

The null hypothesis serves as a benchmark for comparison, while the alternative hypothesis represents the claim researchers seek to support. The process involves collecting data and assessing its compatibility with the null hypothesis through statistical analysis. 

If the observed data significantly deviates from what would be expected under the null hypothesis, researchers may reject the null hypothesis in favor of the alternative, concluding that there is evidence to support their claim. Conversely, if the data does not provide sufficient evidence, the null hypothesis is retained. Hypothesis testing provides a structured framework for drawing conclusions and making informed decisions in scientific research and data analysis.

Just as a start up example, if we want to look at the distribution of age amongst different races

Null Hypothesis (H0): There is no significant difference in the distribution of age among different races.

Alternative Hypothesis (H1): There is a significant difference in the distribution of age among different races.

In this example, the null hypothesis assumes that there is no variation in age distribution across various racial groups, suggesting that any observed differences are due to random chance. The alternative hypothesis, on the other hand, posits that there is a significant difference in the distribution of age among these racial groups. Researchers would conduct a statistical analysis on age data from different races to determine whether the evidence supports rejecting the null hypothesis in favor of the alternative, indicating that there is a meaningful distinction in age distribution based on race.

For the rest of this lab, we want to look at distribution of time of day stopped amongst different races, write the null and alternative hypothesis by yourself below. Refer back to the previous question if you are feeling stuck!

Null Hypothesis: **Your Answer Here**

Alternative Hypothesis: **Your Answer Here**


## 3. Bootstrap

Bootstrapping is a statistical technique that allows us to make educated guesses about a population using only a small sample from that population. It works by repeatedly taking small random samples from the data we have and then using these samples to estimate things like averages, variances, or other statistics, as if we had data for the entire population. This technique helps us understand how uncertain or variable our estimates are and is especially useful when we have limited data.

In the below section we will use the average age as an example and show you how bootstrapping works to find 

Write code that resamples the "ftobama" distribution, then plot a histogram. Be sure to resample the number of rows that exist in the dataset, with replacement.

In [ ]:
# Find number of rows

In [ ]:
# Resample the data

In [ ]:
# Histogram

In [ ]:
# 50th percentile/median

### Question 2

How does the resampled median compare to the original median? Does this result make sense?

Now, define a function titled "bootstrap_median" that takes the original sample, the column name we're concerned with, and the number of resamples as arguments. The function should calculate simulated medians and return them in an array.

In [ ]:
# Define a function "bootstrap_median" with arguments "original_sample", "label", and "replications" 
# that returns an array with the medians found in replications
def bootstrap_median(original_sample, label, replications):
    """Returns an array of bootstrapped sample medians:
    original_sample: table containing the original sample
    label: label of column containing the variable
    replications: number of bootstrap samples
    """
    just_one_column = original_sample.loc[:, label]
    medians = []
    for i in np.arange(replications):
        ...

    return ...

Replicate the bootstrap 10,000 times, then save the results.

In [ ]:
# Resample 10,000 times

## Confidence Interval

In statistics and data analysis, confidence intervals are useful tools that help us measure how certain or uncertain we are about an estimate. They allow us to figure out a likely range for something, like the average of a whole group, based on information from a smaller part of that group. Imagine it like a "guess range" to see how accurate our estimates are.

Similar to the idea of bootstrapping, which helps us figure out the spread of numbers, confidence intervals turn this idea into something practical. They help us say how sure we are that a population number is in a certain range, using information from our sample.

In this section, we'll take a closer look at how confidence intervals work and see why they're important in statistics. They not only help us check the accuracy of our guesses but also guide us in making decisions and understanding how much we can trust our findings from the data

Plot a histogram of the resampled medians, and plot the 95% confidence interval. (hint: to plot the confidence interval, try using the 2.5 percentile and 97.5 percentile values in a numpy array)

In [ ]:
# Plot medians
...
plots.plot(np.array([pd.Series(medians).quantile(q=.025), pd.Series(medians).quantile(q=.975)]), np.array([0, 0]), color='yellow', lw=10, zorder=1)

### Question 3

What can you infer about the likely population median given the above distribution?

Finally, write a simulation that constructs 100 confidence intervals. (Remember to use the 2.5 and 97.5 percentiles!)

In [ ]:
# Construct 100 confidence intervals
left_ends = []
right_ends = []

for i in np.arange(100):
    ...
    
intervals = pd.DataFrame(data={"Left": left_ends, "Right": right_ends})

#### Question 4

Finally, plot 100 confidence intervals (stacked on top of each other). What can you conclude about the median?

In [ ]:
# Plot the confidence intervals
plots.figure(figsize=(8,8))
for i in np.arange(100):
    ends = ...
    plots.plot(ends, np.array([i + 1, i + 1]), color='gold')
plots.xlabel('Median')
plots.ylabel('Replication')
plots.title('Population Median and Intervals of Estimates');

## Forming a Hypothesis

## Confidence Interval

In statistics and data analysis, confidence intervals are useful tools that help us measure how certain or uncertain we are about an estimate. They allow us to figure out a likely range for something, like the average of a whole group, based on information from a smaller part of that group. Imagine it like a "guess range" to see how accurate our estimates are.

Similar to the idea of bootstrapping, which helps us figure out the spread of numbers, confidence intervals turn this idea into something practical. They help us say how sure we are that a population number is in a certain range, using information from our sample.

In this section, we'll take a closer look at how confidence intervals work and see why they're important in statistics. They not only help us check the accuracy of our guesses but also guide us in making decisions and understanding how much we can trust our findings from the data

## Central Limit Theorem (CLT)